In [2]:
import torch
import pandas as pd
import ast
from gensim.models import KeyedVectors

model_path = '../models/GoogleNews-vectors-negative300.bin.gz' #Actualiza el path
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

KeyboardInterrupt: 

In [3]:
df_train = pd.read_csv("../data/NER_SA_csvs/train.csv")
df_val = pd.read_csv("../data/NER_SA_csvs/validation.csv")
df_test = pd.read_csv("../data/NER_SA_csvs/test.csv")

In [39]:
# Encuentra el valor máximo de cada lista de 'ner_tags' y luego encuentra el valor máximo global

max_ner_tag = max(df_train['ner_tags'].apply(lambda x: max(x)))

print(f"El número único máximo de NER tags es: {max_ner_tag}")




El número único máximo de NER tags es: 8
0        7
1        2
2        5
3        7
4        5
        ..
14036    0
14037    0
14038    3
14039    0
14040    3
Name: ner_tags, Length: 14041, dtype: int64


In [38]:
sentences_train = df_train['sentence'].tolist()
sentences_val = df_val['sentence'].tolist()
sentences_test = df_test['sentence'].tolist()

In [50]:

# Cargar el modelo preentrenado de Google News
# Asegúrate de tener el archivo .bin de Word2Vec de Google News

# Función para obtener los embeddings de las palabras en cada frase
def get_embeddings_for_sentence(sentence):
    words = sentence.split()  # Dividimos la frase en palabras
    embeddings = []
    for word in words:
        try:
            # Obtenemos el embedding de cada palabra, si está en el vocabulario del modelo
            embedding = model[word]
            embeddings.append(embedding)
        except KeyError:
            # Si la palabra no está en el vocabulario del modelo, agregamos un vector cero
            embeddings.append([0] * 300)  # 300 es el tamaño del vector en el modelo de Google News
    return embeddings

